In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np 

In [2]:
MILKING_PATH = "Dairy Data/dbo_tblMilkDaily.csv"
STOCKLIST_PATH = "Dairy Data/dbo_tblStocklist.csv"
AILMENTS_PATH = "Dairy Data/Ailments_20220101-20231214.xlsx"
LACTATION_PATH = "Dairy Data/dbo_tblLactation.csv"
ALFLEX_PATH = "Alflex"
ALFLEX_REESULT_PATH = "Alflex_trans"
HERT_TEST_PATH = "Herd Test/HT results 2022-2023.csv"

SAVE_DATA = True

# Milking Data

In [3]:
milking_df = pd.read_csv(MILKING_PATH)

milking_df['MilkDate'] = milking_df['MilkDate'].str.replace(' 0:00:00', '')
milking_df['MilkDate'] = pd.to_datetime(milking_df['MilkDate'], format='%d/%m/%Y')

milking_df['CowID'] = milking_df['CowID'].astype(str)

/Users/leochen/opt/anaconda3/envs/NLP/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (0,8,9,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Drop Column

In [4]:
# Drop column that not use in this project
milking_df = milking_df.drop('GroupNo', axis=1)
milking_df = milking_df.drop('MilkTotal', axis=1)
milking_df = milking_df.drop('StallPM', axis=1)
milking_df = milking_df.drop('StallAM', axis=1)
milking_df = milking_df.drop('TimePM', axis=1)
milking_df = milking_df.drop('TimeAM', axis=1)
milking_df = milking_df.drop('MilkQT', axis=1)
milking_df = milking_df.drop('PMweight', axis=1)
milking_df = milking_df.drop('PMfatpc', axis=1)
milking_df = milking_df.drop('PMprotpc', axis=1)
milking_df = milking_df.drop('PMlactpc', axis=1)
milking_df = milking_df.drop('PMscc', axis=1)
milking_df = milking_df.drop('AMweight', axis=1)
milking_df = milking_df.drop('AMfatpc', axis=1)
milking_df = milking_df.drop('AMprotpc', axis=1)
milking_df = milking_df.drop('AMlactpc', axis=1)
milking_df = milking_df.drop('AMscc', axis=1)
milking_df = milking_df.drop('HerdTestQT', axis=1)

### Focus on Date

In [5]:
# Specify the date range
earliest_date = pd.to_datetime('2022-01-01', format='%Y-%m-%d')
latest_date = pd.to_datetime('2023-12-31', format='%Y-%m-%d')

# Filter the DataFrame based on the date range
milking_df = milking_df[(milking_df['MilkDate'] >= earliest_date) & (milking_df['MilkDate'] <= latest_date)]


### Focus on Elinbank

In [6]:
# read stocklist
stocklist_df = pd.read_csv(STOCKLIST_PATH)

# change data type
stocklist_df['CowID'] = stocklist_df['CowID'].astype(str)
stocklist_df['CowNo'] = stocklist_df['CowNo'].astype(int)
stocklist_df['ShireFarm'] = stocklist_df['ShireFarm'].astype(str)

# get elinbank_stocklist
elinbank_stocklist_df = stocklist_df[stocklist_df['ShireFarm'] == '808183']
elinbank_stocklist_df

,CowID,ShireFarm,CowNo,BirthDate,Dam,Shire
119,206004399,808183,13359,NaN,NaN,NaN
120,206004408,808183,19618,NaN,NaN,NaN
121,206004410,808183,19668,NaN,NaN,NaN
122,206004413,808183,9320,NaN,NaN,NaN
123,206004415,808183,19683,NaN,NaN,NaN
...,...,...,...,...,...,...
38343,Y10ST0135,808183,693,13/9/2010 0:00:00,505009951,ZIRCON
38344,Y10ST0136,808183,666,13/9/2010 0:00:00,505009988,ZIRCON
38524,Y11ST0007,808183,1203,22/7/2011 0:00:00,504000538,!FRIES
38525,Y11ST0011,808183,1206,NaN,209021293,ORANA


In [7]:
# filter milking data
milking_df = milking_df[milking_df['CowID'].isin(elinbank_stocklist_df['CowID'])]

milking_cow_ids = milking_df["CowID"]
milking_cow_ids = milking_cow_ids.drop_duplicates()
print("Number of Cows:", len(milking_cow_ids))

milking_df

Number of Cows: 643


,CowID,MilkDate,MilkPM,MilkAM,DurationPM,DurationAM,PFlowPM,PFlowAM
48082,207019964,2022-01-01,7.02,10.84,7.41,9.36,2.3,2.0
48083,207019964,2022-01-02,7.16,9.99,8.63,9.38,2.3,2.0
48084,207019964,2022-01-03,4.33,7.10,6.33,8.78,1.4,1.7
48085,207019964,2022-01-04,7.53,3.96,9.00,6.70,2.0,1.1
48086,207019964,2022-01-05,8.39,6.81,9.30,8.28,2.1,1.6
...,...,...,...,...,...,...,...,...
4422356,Y10ST0117,2023-03-10,6.42,9.79,4.76,6.16,2.3,2.3
4422357,Y10ST0117,2023-03-11,5.43,10.96,5.01,8.20,2.0,2.8
4422358,Y10ST0117,2023-03-12,5.77,8.86,4.33,6.75,2.2,2.2
4422359,Y10ST0117,2023-03-13,6.09,8.66,4.61,6.86,2.1,2.0


### Seperate AM and PM

In [8]:
# Melt the DataFrame to reshape it
melted_milking_df = pd.melt(milking_df, id_vars=['CowID', 'MilkDate'], var_name='Variable', value_name='Value')

# Extract the time and measurement type
melted_milking_df[['Measurement', 'Time']] = melted_milking_df['Variable'].str.extract(r'([A-Za-z]+)(AM|PM)')

# Pivot the DataFrame to get the desired format
milking_df = melted_milking_df.pivot_table(index=['CowID', 'MilkDate', 'Time'], columns='Measurement', values='Value', aggfunc='first').reset_index()

# Rename the columns
milking_df.columns.name = None  # Remove the name of the columns index
milking_df = milking_df.rename(columns={'A': 'MilkA', 'P': 'MilkP', 'DurationA': 'DurationA', 'DurationP': 'DurationP', 'PFlowA': 'PFlowA', 'PFlowP': 'PFlowP'})

milking_df

,CowID,MilkDate,Time,Duration,Milk,PFlow
0,207019964,2022-01-01,AM,9.36,10.84,2.0
1,207019964,2022-01-01,PM,7.41,7.02,2.3
2,207019964,2022-01-02,AM,9.38,9.99,2.0
3,207019964,2022-01-02,PM,8.63,7.16,2.3
4,207019964,2022-01-03,AM,8.78,7.10,1.7
...,...,...,...,...,...,...
492197,Y10ST0117,2023-03-12,PM,4.33,5.77,2.2
492198,Y10ST0117,2023-03-13,AM,6.86,8.66,2.0
492199,Y10ST0117,2023-03-13,PM,4.61,6.09,2.1
492200,Y10ST0117,2023-03-14,AM,7.88,10.02,2.5


# Ailment Data (Mastitis)

In [9]:
ailments_df = pd.read_excel(AILMENTS_PATH)

ailments_df['Animal Number'] = ailments_df['Animal Number'].astype(int)
ailments_df['Event Date'] = pd.to_datetime(ailments_df['Event Date'], format='%d/%m/%Y')

In [10]:
ailments_df = ailments_df.drop('Location', axis=1)
ailments_df = ailments_df.drop('Treatment', axis=1)
ailments_df = ailments_df.drop('Drugs', axis=1)
ailments_df = ailments_df.drop('Comment', axis=1)
ailments_df = ailments_df.drop('Category', axis=1)
ailments_df = ailments_df.drop('User', axis=1)
ailments_df = ailments_df.drop('Group', axis=1)
ailments_df = ailments_df.drop('Exit Date', axis=1)
ailments_df = ailments_df.drop('Last Calving Date At Event', axis=1)
ailments_df = ailments_df.drop('Gestation Days at Event', axis=1)
ailments_df = ailments_df.drop('Reproduction Status at Event', axis=1)

In [11]:
# ailments_mastitis_df = ailments_df[ailments_df['Diagnosis'].str.contains('mastitis', case=False, na=False)]
ailments_mastitis_df = ailments_df[ailments_df['Diagnosis'].str.contains('mastitis', case=False, na=False) & ~ailments_df['Diagnosis'].str.contains('MastitisPrevent', case=False, na=False)]

ailments_cow_ids = ailments_mastitis_df['Animal Number']
ailments_cow_ids = ailments_cow_ids.drop_duplicates()

print("Number of Cows with Mastitis:", len(ailments_cow_ids))
ailments_mastitis_df

Number of Cows with Mastitis: 243


,Animal Number,Event Date,Diagnosis,Lactation Number,DIM at Event
3,552,2022-08-30,Mastitis,11,34.0
5,552,2022-02-21,Mastitis,10,191.0
6,552,2022-04-14,Mastitis,10,243.0
12,645,2022-01-13,Mastitis,8,138.0
14,645,2022-02-10,Mastitis,8,166.0
...,...,...,...,...,...
8822,7732,2023-11-19,Mastitis,1,88.0
8849,7736,2023-07-07,Mastitis,1,8.0
8857,7737,2023-09-04,Mastitis,1,5.0
8870,7740,2023-08-04,Mastitis,1,14.0


In [12]:
# Merge dataframes on 'Animal Number' and 'CowNo'
ailments_mastitis_df = pd.merge(ailments_mastitis_df, elinbank_stocklist_df, left_on='Animal Number', right_on='CowNo', how='left')

# Rename the merged column to 'CowID'
ailments_mastitis_df.rename(columns={'CowID_y': 'CowID'}, inplace=True)

# Drop unnecessary columns from the merged dataframe
ailments_mastitis_df.drop(['ShireFarm', 'CowNo', 'Dam', 'Shire', 'BirthDate'], axis=1, inplace=True)

# Display the updated dataframmerged_df
ailments_mastitis_df

,Animal Number,Event Date,Diagnosis,Lactation Number,DIM at Event,CowID
0,552,2022-08-30,Mastitis,11,34.0,211012544
1,552,2022-02-21,Mastitis,10,191.0,211012544
2,552,2022-04-14,Mastitis,10,243.0,211012544
3,645,2022-01-13,Mastitis,8,138.0,Y10ST0079
4,645,2022-02-10,Mastitis,8,166.0,Y10ST0079
...,...,...,...,...,...,...
606,7732,2023-11-19,Mastitis,1,88.0,221035484
607,7736,2023-07-07,Mastitis,1,8.0,221035488
608,7737,2023-09-04,Mastitis,1,5.0,221035489
609,7740,2023-08-04,Mastitis,1,14.0,221035492


In [13]:
# Initialize a new column 'Label' in the Milk DataFrame with default value 0
milking_df['Mastitis'] = 0
ailments_mastitis_df['Label'] = 0

# Iterate through each row in the Mastitis DataFrame
for _, mastitis_row in ailments_mastitis_df.iterrows():
    cow_id = mastitis_row['CowID']
    obs_date = mastitis_row['Event Date']
    
    # Check if there is a corresponding row in the Milk DataFrame
    matching_rows = milking_df[(milking_df['CowID'] == cow_id) & (milking_df['MilkDate'] == obs_date)]
    
    # Update the 'Label' column in the Milk DataFrame to 1 for matching rows
    milking_df.loc[matching_rows.index, 'Mastitis'] = 1 

    if not matching_rows.empty:
        ailments_mastitis_df.at[mastitis_row.name, 'Label'] = 1

In [14]:
mastitis_count = milking_df['Mastitis'].sum()
print("Numbers of Label in milking_df: ", mastitis_count)

Numbers of Label in milking_df:  1139


# Lactation

In [15]:
lactation_df = pd.read_csv(LACTATION_PATH)

lactation_df['LactDate'] = pd.to_datetime(lactation_df['LactDate'], format='%d/%m/%Y %H:%M:%S')

In [16]:
lactation_df = lactation_df.drop('LactEvent', axis=1)
lactation_df = lactation_df.drop('LactEventNotes', axis=1)
lactation_df = lactation_df.drop('LactQT', axis=1)

In [17]:
lactation_df = lactation_df[lactation_df['Category'] == 'Calved']

In [18]:
# Sort lactation_df by 'CowID' and 'LactDate'
lactation_df = lactation_df.sort_values(by=['CowID', 'LactDate'])

# Add a new column 'LactationNO' to lactation_df
lactation_df['LactationNo'] = lactation_df.groupby('CowID').cumcount()
lactation_df['LactationNo'] = lactation_df['LactationNo'] + 1
lactation_df

,CowID,LactDate,Category,LactationNo
43,200001174,2001-08-07,Calved,1
45,200001174,2002-08-08,Calved,2
47,200001174,2003-10-05,Calved,3
49,200001174,2004-09-23,Calved,4
51,200001174,2006-03-15,Calved,5
...,...,...,...,...
122907,Z219W0452,2022-08-29,Calved,1
122908,Z229W0127,2022-07-31,Calved,1
122909,Z229W0128,2022-08-01,Calved,1
122910,Z229W0129,2022-08-15,Calved,1


In [19]:
# Merge milking_df with matching lactation_df rows based on 'CowID'
merged_df = pd.merge(milking_df, lactation_df, on='CowID', how='left')

# Filter rows where 'LactDate' is later than 'MilkDate'
# todo: may have issue
filtered_rows = merged_df[merged_df['LactDate'] < merged_df['MilkDate']]

# Group by the original milking_df index and get the maximum 'LactDate' for each group
latest_lact_dates = filtered_rows.groupby(["CowID", "MilkDate", "Time"])['LactDate'].max()
try:
    lact_no = filtered_rows.groupby(["CowID", "MilkDate", "Time"])['LactationNo_y'].max()
except:
    lact_no = filtered_rows.groupby(["CowID", "MilkDate", "Time"])['LactationNo'].max()

for index in milking_df.index:
    cow_id = milking_df.at[index, 'CowID']
    milk_date = milking_df.at[index, 'MilkDate']
    time = milking_df.at[index, 'Time']

    # Check if the index exists in latest_lact_dates
    if (cow_id, milk_date, time) in latest_lact_dates.index:
        targrt_date = latest_lact_dates.loc[(cow_id, milk_date, time)]
        milking_df.at[index, 'LacDay'] = latest_lact_dates.loc[(cow_id, milk_date, time)]
        milking_df.at[index, 'LactationNo'] = lact_no.loc[(cow_id, milk_date, time)]
    else:
        milking_df.at[index, 'LacDay'] = pd.NaT
        milking_df.at[index, 'LactationNo'] = pd.NaT

milking_df['DIM'] = (milking_df['MilkDate'] - milking_df['LacDay']).dt.days
milking_df

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
0,207019964,2022-01-01,AM,9.36,10.84,2.0,0,2021-07-08,3.0,177.0
1,207019964,2022-01-01,PM,7.41,7.02,2.3,0,2021-07-08,3.0,177.0
2,207019964,2022-01-02,AM,9.38,9.99,2.0,0,2021-07-08,3.0,178.0
3,207019964,2022-01-02,PM,8.63,7.16,2.3,0,2021-07-08,3.0,178.0
4,207019964,2022-01-03,AM,8.78,7.10,1.7,0,2021-07-08,3.0,179.0
...,...,...,...,...,...,...,...,...,...,...
492197,Y10ST0117,2023-03-12,PM,4.33,5.77,2.2,0,2021-07-10,3.0,610.0
492198,Y10ST0117,2023-03-13,AM,6.86,8.66,2.0,0,2021-07-10,3.0,611.0
492199,Y10ST0117,2023-03-13,PM,4.61,6.09,2.1,0,2021-07-10,3.0,611.0
492200,Y10ST0117,2023-03-14,AM,7.88,10.02,2.5,0,2021-07-10,3.0,612.0


In [20]:
filtered_rows

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LactDate,Category,LactationNo
0,207019964,2022-01-01,AM,9.36,10.84,2.0,0,2019-07-14,Calved,1.0
1,207019964,2022-01-01,AM,9.36,10.84,2.0,0,2020-07-27,Calved,2.0
2,207019964,2022-01-01,AM,9.36,10.84,2.0,0,2021-07-08,Calved,3.0
4,207019964,2022-01-01,PM,7.41,7.02,2.3,0,2019-07-14,Calved,1.0
5,207019964,2022-01-01,PM,7.41,7.02,2.3,0,2020-07-27,Calved,2.0
...,...,...,...,...,...,...,...,...,...,...
1446121,Y10ST0117,2023-03-14,AM,7.88,10.02,2.5,0,2020-07-05,Calved,2.0
1446122,Y10ST0117,2023-03-14,AM,7.88,10.02,2.5,0,2021-07-10,Calved,3.0
1446123,Y10ST0117,2023-03-14,PM,4.90,6.65,2.3,0,2019-08-02,Calved,1.0
1446124,Y10ST0117,2023-03-14,PM,4.90,6.65,2.3,0,2020-07-05,Calved,2.0


### Remove CowID with missing DIM

In [21]:
# Find CowID containing any rows where DIM is nan
# invalid_cow_ids = milking_df[(milking_df['DIM'].isna()) | (milking_df['DIM'] > 365)]['CowID'].unique()
invalid_cow_ids = milking_df[milking_df['DIM'].isna()]['CowID'].unique()

# Print the number of rows being deleted
print("Number of CowID deleted:", len(invalid_cow_ids))

# Delete CowID contains nan DIM
milking_df = milking_df[~milking_df['CowID'].isin(invalid_cow_ids)]

# Delete rows DIM > 365
milking_df = milking_df[milking_df['DIM'] <= 365]

milking_cow_ids = milking_df["CowID"]
milking_cow_ids = milking_cow_ids.drop_duplicates()
print("Number of Cows:", len(milking_cow_ids))
print("Number of rows:", milking_df.shape[0])

Number of CowID deleted: 31
Number of Cows: 601
Number of rows: 415659


In [22]:
milking_df

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
0,207019964,2022-01-01,AM,9.36,10.84,2.0,0,2021-07-08,3.0,177.0
1,207019964,2022-01-01,PM,7.41,7.02,2.3,0,2021-07-08,3.0,177.0
2,207019964,2022-01-02,AM,9.38,9.99,2.0,0,2021-07-08,3.0,178.0
3,207019964,2022-01-02,PM,8.63,7.16,2.3,0,2021-07-08,3.0,178.0
4,207019964,2022-01-03,AM,8.78,7.10,1.7,0,2021-07-08,3.0,179.0
...,...,...,...,...,...,...,...,...,...,...
491703,Y10ST0117,2022-07-08,PM,10.85,11.05,2.3,0,2021-07-10,3.0,363.0
491704,Y10ST0117,2022-07-09,AM,13.46,15.49,1.8,0,2021-07-10,3.0,364.0
491705,Y10ST0117,2022-07-09,PM,7.93,11.24,2.6,0,2021-07-10,3.0,364.0
491706,Y10ST0117,2022-07-10,AM,13.26,16.24,3.0,0,2021-07-10,3.0,365.0


# Handling Missing

In [23]:
# Fill missing PFlow values based on Milk/Duration
missing_pflow_mask = milking_df['PFlow'].isna() | (milking_df['PFlow'] == 0)
valid_data_mask = (milking_df['Milk'] != 0) & (milking_df['Duration'] != 0)
milking_df.loc[missing_pflow_mask & valid_data_mask, 'PFlow'] = milking_df['Milk'] / milking_df['Duration']

missing_duration_mask = milking_df['Duration'].isna() | (milking_df['Duration'] == 0)
valid_data_mask = (milking_df['Milk'] != 0) & (milking_df['PFlow'] != 0)
milking_df.loc[missing_duration_mask & valid_data_mask, 'Duration'] = milking_df['Milk'] / milking_df['PFlow']

In [24]:
# Filter rows with Duration, Milk, or PFlow equals 0 or missing
rows_with_nan = milking_df[(milking_df['Duration'] == 0) | (milking_df['Milk'] == 0) | (milking_df['PFlow'] == 0) |
                           milking_df['Duration'].isnull() | milking_df['Milk'].isnull() | milking_df['PFlow'].isnull()]

rows_with_nan

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
242,208000988,2022-01-15,PM,9.73,NaN,NaN,0,2021-10-10,2.0,97.0
1024,211012544,2022-03-30,AM,6.98,NaN,NaN,0,2021-08-14,3.0,228.0
1141,211012544,2022-07-31,PM,9.46,NaN,NaN,0,2021-08-14,3.0,351.0
1311,213012748,2022-07-31,PM,4.51,NaN,NaN,1,2022-07-23,3.0,8.0
1645,213012748,2023-01-14,PM,5.10,NaN,NaN,0,2022-07-23,3.0,175.0
...,...,...,...,...,...,...,...,...,...,...
489575,221036190,2023-11-11,PM,0.80,NaN,NaN,0,2023-08-19,1.0,84.0
491280,Y10ST0112,2023-10-31,AM,NaN,14.04,NaN,0,2023-09-16,4.0,45.0
491281,Y10ST0112,2023-10-31,PM,NaN,12.13,NaN,0,2023-09-16,4.0,45.0
491282,Y10ST0112,2023-11-01,AM,NaN,15.61,NaN,0,2023-09-16,4.0,46.0


In [25]:
# Find CowID containing any rows where Mastitis=1
mastitis_cowid_list = milking_df.loc[milking_df['Mastitis'] == 1, 'CowID'].unique()

# Find CowID containing any rows where both Duration and PFlow are nan
nan_duration_pflow_cowid_list = milking_df.loc[(milking_df['Duration'].isna() & milking_df['PFlow'].isna()) | milking_df['Milk'].isna(), 'CowID'].unique()

# Find CowID that contain any rows where both Duration and PFlow are nan
# but these CowID do not appear in the list with Mastitis=1
final_cowid_list = set(nan_duration_pflow_cowid_list) - set(mastitis_cowid_list)

# Filter rows to be deleted
rows_to_delete = milking_df[milking_df['CowID'].isin(final_cowid_list)]

# Print the number of rows being deleted
print("Number of CowID deleted:", len(final_cowid_list))
print("Number of rows deleted:", len(rows_to_delete))

milking_df = milking_df[~milking_df['CowID'].isin(final_cowid_list)]

Number of CowID deleted: 188
Number of rows deleted: 155292


In [26]:
# Filter rows with Duration, Milk, or PFlow equals 0 or missing
rows_with_nan = milking_df[(milking_df['Duration'] == 0) | (milking_df['Milk'] == 0) | (milking_df['PFlow'] == 0) |
                           milking_df['Duration'].isnull() | milking_df['Milk'].isnull() | milking_df['PFlow'].isnull()]

rows_with_nan

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
242,208000988,2022-01-15,PM,9.73,NaN,NaN,0,2021-10-10,2.0,97.0
1024,211012544,2022-03-30,AM,6.98,NaN,NaN,0,2021-08-14,3.0,228.0
1141,211012544,2022-07-31,PM,9.46,NaN,NaN,0,2021-08-14,3.0,351.0
1311,213012748,2022-07-31,PM,4.51,NaN,NaN,1,2022-07-23,3.0,8.0
1645,213012748,2023-01-14,PM,5.10,NaN,NaN,0,2022-07-23,3.0,175.0
...,...,...,...,...,...,...,...,...,...,...
478613,221035453,2023-10-01,PM,4.16,NaN,NaN,0,2023-08-11,1.0,51.0
491280,Y10ST0112,2023-10-31,AM,NaN,14.04,NaN,0,2023-09-16,4.0,45.0
491281,Y10ST0112,2023-10-31,PM,NaN,12.13,NaN,0,2023-09-16,4.0,45.0
491282,Y10ST0112,2023-11-01,AM,NaN,15.61,NaN,0,2023-09-16,4.0,46.0


In [27]:
# Fill missing Milk values with the average Milk for each CowID
milking_df['Milk'] = milking_df['Milk'].fillna(method='ffill').fillna(method='bfill')

/var/folders/sf/xq24qhg94mvdr757gpyj945m0000gn/T/ipykernel_28912/896591868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milking_df['Milk'] = milking_df['Milk'].fillna(method='ffill').fillna(method='bfill')


In [28]:
# Fill missing PFlow values based on Milk/Duration
missing_pflow_mask = milking_df['PFlow'].isna() | (milking_df['PFlow'] == 0)
valid_data_mask = (milking_df['Milk'] != 0) & (milking_df['Duration'] != 0)
milking_df.loc[missing_pflow_mask & valid_data_mask, 'PFlow'] = milking_df['Milk'] / milking_df['Duration']

missing_duration_mask = milking_df['Duration'].isna() | (milking_df['Duration'] == 0)
valid_data_mask = (milking_df['Milk'] != 0) & (milking_df['PFlow'] != 0)
milking_df.loc[missing_duration_mask & valid_data_mask, 'Duration'] = milking_df['Milk'] / milking_df['PFlow']

/Users/leochen/opt/anaconda3/envs/NLP/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/leochen/opt/anaconda3/envs/NLP/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [29]:
# Fill missing Milk values with the average Milk for each CowID
milking_df['Duration'] = milking_df['Duration'].fillna(method='ffill').fillna(method='bfill')

/var/folders/sf/xq24qhg94mvdr757gpyj945m0000gn/T/ipykernel_28912/197492337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milking_df['Duration'] = milking_df['Duration'].fillna(method='ffill').fillna(method='bfill')


In [30]:
# Fill missing PFlow values based on Milk/Duration
missing_pflow_mask = milking_df['PFlow'].isna() | (milking_df['PFlow'] == 0)
valid_data_mask = (milking_df['Milk'] != 0) & (milking_df['Duration'] != 0)
milking_df.loc[missing_pflow_mask & valid_data_mask, 'PFlow'] = milking_df['Milk'] / milking_df['Duration']

/Users/leochen/opt/anaconda3/envs/NLP/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [31]:
# Make sure no missing data
rows_with_nan = milking_df[(milking_df['Duration'] == 0) | (milking_df['Milk'] == 0) | (milking_df['PFlow'] == 0) |
                           milking_df['Duration'].isnull() | milking_df['Milk'].isnull() | milking_df['PFlow'].isnull()]

rows_with_nan

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM


### DownSampling: preserve cowID only with mastitis

In [32]:
# Filter the DataFrame
mastitis_cow_ids = milking_df[milking_df['Mastitis'] == 1]['CowID'].unique()
milking_df = milking_df[milking_df['CowID'].isin(mastitis_cow_ids)]

milking_df

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
213,208000988,2022-01-01,AM,8.66,20.63,3.2,0,2021-10-10,2.0,83.0
214,208000988,2022-01-01,PM,5.10,11.60,3.6,0,2021-10-10,2.0,83.0
215,208000988,2022-01-02,AM,8.53,19.72,3.1,0,2021-10-10,2.0,84.0
216,208000988,2022-01-02,PM,5.43,11.11,3.3,0,2021-10-10,2.0,84.0
217,208000988,2022-01-03,AM,8.60,19.29,3.1,0,2021-10-10,2.0,85.0
...,...,...,...,...,...,...,...,...,...,...
491321,Y10ST0112,2023-11-20,PM,3.51,11.53,5.0,0,2023-09-16,4.0,65.0
491322,Y10ST0112,2023-11-21,AM,4.71,12.67,4.0,0,2023-09-16,4.0,66.0
491323,Y10ST0112,2023-11-21,PM,4.08,10.60,4.3,0,2023-09-16,4.0,66.0
491324,Y10ST0112,2023-11-22,AM,4.93,13.62,4.3,0,2023-09-16,4.0,67.0


### Overall

In [33]:
milking_cow_ids = milking_df["CowID"]
milking_cow_ids = milking_cow_ids.drop_duplicates()
mastitis_count = milking_df['Mastitis'].sum()

print("Number of Cows:", len(milking_cow_ids))
print("Number of rows:", milking_df.shape[0])
print("Numbers of Label in milking_df: ", mastitis_count)

Number of Cows: 211
Number of rows: 164693
Numbers of Label in milking_df:  1021


### Check continuous

In [34]:
milking_df_am = milking_df[milking_df['Time'] == 'AM']
milking_df_pm = milking_df[milking_df['Time'] == 'PM']

print("Number of rows in milking_df_am:", milking_df_am.shape[0])
print("Number of rows in milking_df_pm:", milking_df_pm.shape[0])

Number of rows in milking_df_am: 82351
Number of rows in milking_df_pm: 82342


In [35]:
milking_df_pm

,CowID,MilkDate,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
214,208000988,2022-01-01,PM,5.10,11.60,3.6,0,2021-10-10,2.0,83.0
216,208000988,2022-01-02,PM,5.43,11.11,3.3,0,2021-10-10,2.0,84.0
218,208000988,2022-01-03,PM,3.70,8.06,3.4,0,2021-10-10,2.0,85.0
220,208000988,2022-01-04,PM,6.00,10.41,3.2,0,2021-10-10,2.0,86.0
222,208000988,2022-01-05,PM,5.93,11.57,3.4,0,2021-10-10,2.0,87.0
...,...,...,...,...,...,...,...,...,...,...
491317,Y10ST0112,2023-11-18,PM,3.91,12.59,5.5,0,2023-09-16,4.0,63.0
491319,Y10ST0112,2023-11-19,PM,4.13,9.62,4.1,0,2023-09-16,4.0,64.0
491321,Y10ST0112,2023-11-20,PM,3.51,11.53,5.0,0,2023-09-16,4.0,65.0
491323,Y10ST0112,2023-11-21,PM,4.08,10.60,4.3,0,2023-09-16,4.0,66.0


### Fill in missing date [PM]

In [36]:
# Group milking_df by CowID and LactationNo
grouped = milking_df_pm.groupby(['CowID', 'LactationNo'])

# Initialize a list to store modified groups
modified_groups = []

# Iterate through each group
for (cowid, lactationno), group in grouped:
    # Sort the group by MilkDate
    group = group.sort_values(by='MilkDate')

    # Set MilkDate as the index
    group.set_index('MilkDate', inplace=True)

    # Resample the group to fill in missing dates with a maximum gap of 5 days
    resampled_group = group.resample('D').ffill(limit=5)  # 會填5個

    # Reset the index to turn MilkDate back into a column
    resampled_group.reset_index(inplace=True)

    # Interpolate the new rows
    resampled_group['Duration'] = resampled_group['Duration'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Milk'] = resampled_group['Milk'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['PFlow'] = resampled_group['PFlow'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Mastitis'] = resampled_group['Mastitis'].fillna(0)
    resampled_group['LacDay'] = resampled_group['LacDay'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['LactationNo'] = resampled_group['LactationNo'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['DIM'] = (resampled_group['MilkDate'] - resampled_group['LacDay']).dt.days

    # Workaround
    resampled_group.dropna(inplace=True)

    resampled_group.reset_index(inplace=True)

    # Remove rows where the difference between consecutive MilkDate values is greater than 5 days
    milkdate_diff = resampled_group['MilkDate'].diff().dt.days
    milkdate_diff = milkdate_diff.fillna(1)
    rows_to_remove = milkdate_diff > 1
    if rows_to_remove.any():
        index_to_remove = rows_to_remove[rows_to_remove == True].index[0]  # Find the first index to remove
        resampled_group = resampled_group.iloc[:index_to_remove]  # Remove rows beyond the first index to remove

    # Append the modified group to the list
    modified_groups.append(resampled_group)

# Combine all modified groups into a single DataFrame
new_milking_df_pm = pd.concat(modified_groups)

# Print the new DataFrame
new_milking_df_pm


,index,MilkDate,CowID,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
0,0,2022-01-01,208000988,PM,5.10,11.60,3.6,0.0,2021-10-10,2.0,83
1,1,2022-01-02,208000988,PM,5.43,11.11,3.3,0.0,2021-10-10,2.0,84
2,2,2022-01-03,208000988,PM,3.70,8.06,3.4,0.0,2021-10-10,2.0,85
3,3,2022-01-04,208000988,PM,6.00,10.41,3.2,0.0,2021-10-10,2.0,86
4,4,2022-01-05,208000988,PM,5.93,11.57,3.4,0.0,2021-10-10,2.0,87
...,...,...,...,...,...,...,...,...,...,...,...
62,62,2023-11-18,Y10ST0112,PM,3.91,12.59,5.5,0.0,2023-09-16,4.0,63
63,63,2023-11-19,Y10ST0112,PM,4.13,9.62,4.1,0.0,2023-09-16,4.0,64
64,64,2023-11-20,Y10ST0112,PM,3.51,11.53,5.0,0.0,2023-09-16,4.0,65
65,65,2023-11-21,Y10ST0112,PM,4.08,10.60,4.3,0.0,2023-09-16,4.0,66


In [37]:
# Group milking_df by CowID and LactationNo
grouped = milking_df_am.groupby(['CowID', 'LactationNo'])

# Initialize a list to store modified groups
modified_groups = []

# Iterate through each group
for (cowid, lactationno), group in grouped:
    # Sort the group by MilkDate
    group = group.sort_values(by='MilkDate')

    # Set MilkDate as the index
    group.set_index('MilkDate', inplace=True)

    # Resample the group to fill in missing dates with a maximum gap of 5 days
    resampled_group = group.resample('D').ffill(limit=5)  # 會填5個

    # Reset the index to turn MilkDate back into a column
    resampled_group.reset_index(inplace=True)

    # Interpolate the new rows
    resampled_group['Duration'] = resampled_group['Duration'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Milk'] = resampled_group['Milk'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['PFlow'] = resampled_group['PFlow'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Mastitis'] = resampled_group['Mastitis'].fillna(0)
    resampled_group['LacDay'] = resampled_group['LacDay'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['LactationNo'] = resampled_group['LactationNo'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['DIM'] = (resampled_group['MilkDate'] - resampled_group['LacDay']).dt.days

    # Workaround
    resampled_group.dropna(inplace=True)

    resampled_group.reset_index(inplace=True)

    # Remove rows where the difference between consecutive MilkDate values is greater than 5 days
    milkdate_diff = resampled_group['MilkDate'].diff().dt.days
    milkdate_diff = milkdate_diff.fillna(1)
    rows_to_remove = milkdate_diff > 1
    if rows_to_remove.any():
        index_to_remove = rows_to_remove[rows_to_remove == True].index[0]  # Find the first index to remove
        resampled_group = resampled_group.iloc[:index_to_remove]  # Remove rows beyond the first index to remove

    # Append the modified group to the list
    modified_groups.append(resampled_group)

# Combine all modified groups into a single DataFrame
new_milking_df_am = pd.concat(modified_groups)

# Print the new DataFrame
new_milking_df_am


,index,MilkDate,CowID,Time,Duration,Milk,PFlow,Mastitis,LacDay,LactationNo,DIM
0,0,2022-01-01,208000988,AM,8.66,20.63,3.2,0.0,2021-10-10,2.0,83
1,1,2022-01-02,208000988,AM,8.53,19.72,3.1,0.0,2021-10-10,2.0,84
2,2,2022-01-03,208000988,AM,8.60,19.29,3.1,0.0,2021-10-10,2.0,85
3,3,2022-01-04,208000988,AM,6.75,15.71,3.1,0.0,2021-10-10,2.0,86
4,4,2022-01-05,208000988,AM,7.28,16.94,3.0,0.0,2021-10-10,2.0,87
...,...,...,...,...,...,...,...,...,...,...,...
62,62,2023-11-18,Y10ST0112,AM,5.13,15.17,4.9,0.0,2023-09-16,4.0,63
63,63,2023-11-19,Y10ST0112,AM,4.68,14.52,4.9,0.0,2023-09-16,4.0,64
64,64,2023-11-20,Y10ST0112,AM,5.03,13.47,4.6,0.0,2023-09-16,4.0,65
65,65,2023-11-21,Y10ST0112,AM,4.71,12.67,4.0,0.0,2023-09-16,4.0,66


### Check continuous

In [38]:
# Group milking_df by CowID
grouped = new_milking_df_am.groupby(['CowID', 'LactationNo'])

# Initialize a list to store CowID without continuous MilkDate
cowids_without_continuous_dates = []

# Iterate through each group
for (cowid, lactationno), group in grouped:
    # Sort the group by MilkDate
    group = group.sort_values(by='MilkDate')

    # Check if MilkDate values are continuous
    milkdates_diff = group['MilkDate'].diff().dt.days

    # workaround: the first value always nan
    milkdates_diff = milkdates_diff.fillna(1)

    if (milkdates_diff != 1).any():        
        print(cowid)
        print(milkdates_diff[milkdates_diff != 1])
        cowids_without_continuous_dates.append(cowid)

# Print CowID without continuous MilkDate
print(len(cowids_without_continuous_dates))

0


### Delete 14 days after Mastitis

In [39]:
milking_cow_ids = new_milking_df_am["CowID"]
milking_cow_ids = milking_cow_ids.drop_duplicates()
mastitis_count = new_milking_df_am['Mastitis'].sum()

print("Number of Cows:", len(milking_cow_ids))
print("Number of rows:", new_milking_df_am.shape[0])
print("Numbers of Label in milking_df: ", mastitis_count)

Number of Cows: 211
Number of rows: 82228
Numbers of Label in milking_df:  507.0


In [40]:
grouped = new_milking_df_am.groupby(['CowID', 'LactationNo'])

new_groups = []

for (cowid, lactationno), group in grouped:
    mastitis_indices = group[group['Mastitis'] == 1].index

    for idx in mastitis_indices:
        # 计算需要删除的行的结束索引
        end_index = min(idx + 15, len(group))
        # 删除索引之后的14行（不包括索引本身）
        group.drop(group.index[idx+1:end_index], inplace=True)
    new_groups.append(group)

new_milking_df_am = pd.concat(new_groups)
new_milking_df_am.reset_index(drop=True, inplace=True)

In [41]:
grouped = new_milking_df_pm.groupby(['CowID', 'LactationNo'])

new_groups = []

for (cowid, lactationno), group in grouped:
    mastitis_indices = group[group['Mastitis'] == 1].index

    for idx in mastitis_indices:
        # 计算需要删除的行的结束索引
        end_index = min(idx + 15, len(group))
        # 删除索引之后的14行（不包括索引本身）
        group.drop(group.index[idx+1:end_index], inplace=True)
    new_groups.append(group)

new_milking_df_pm = pd.concat(new_groups)
new_milking_df_pm.reset_index(drop=True, inplace=True)

In [42]:
milking_cow_ids = new_milking_df_am["CowID"]
milking_cow_ids = milking_cow_ids.drop_duplicates()
mastitis_count = new_milking_df_am['Mastitis'].sum()

print("Number of Cows:", len(milking_cow_ids))
print("Number of rows:", new_milking_df_am.shape[0])
print("Numbers of Label in milking_df: ", mastitis_count)

Number of Cows: 211
Number of rows: 76199
Numbers of Label in milking_df:  399.0


### Add five day mean, std and finalize milking data

In [43]:
# Group milking_df by CowID and LactationNo
grouped = new_milking_df_am.groupby(['CowID', 'LactationNo'])

# Initialize a list to store modified groups
final_groups = []
window_size = 5

# Iterate through each group
for (cowid, lactationno), group in grouped:

    group['Mean_Milk'] = group['Milk'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Milk'] = group['Milk'].rolling(window=window_size, min_periods=1).std()

    group['Mean_Duration'] = group['Duration'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Duration'] = group['Duration'].rolling(window=window_size, min_periods=1).std()

    group['Mean_Flow'] = group['PFlow'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Flow'] = group['PFlow'].rolling(window=window_size, min_periods=1).std()

    # delete the first five rows
    group = group.iloc[window_size-1:]
    final_groups.append(group)

# Combine all modified groups into a single DataFrame
final_milking_df_am = pd.concat(final_groups)

final_milking_df_am = final_milking_df_am.drop('LacDay', axis=1)
final_milking_df_am = final_milking_df_am.drop('LactationNo', axis=1)
final_milking_df_am = final_milking_df_am.drop('Time', axis=1)
# final_milking_df_am = final_milking_df_am.drop('MilkDate', axis=1)
final_milking_df_am = final_milking_df_am.drop('index', axis=1)

final_milking_df_am.reset_index(drop=True, inplace=True)

# Print the new DataFrame
final_milking_df_am

,MilkDate,CowID,Duration,Milk,PFlow,Mastitis,DIM,Mean_Milk,Std_Milk,Mean_Duration,Std_Duration,Mean_Flow,Std_Flow
0,2022-01-05,208000988,7.28,16.94,3.0,0.0,87,18.458,2.052942,7.964,0.887542,3.10,0.070711
1,2022-01-06,208000988,8.41,20.27,3.7,0.0,88,18.386,1.962022,7.914,0.844529,3.20,0.282843
2,2022-01-07,208000988,8.08,19.93,3.4,0.0,89,18.428,1.999605,7.824,0.784302,3.26,0.288097
3,2022-01-08,208000988,7.91,19.06,3.1,0.0,90,18.382,1.977339,7.686,0.665304,3.26,0.288097
4,2022-01-09,208000988,10.43,19.41,2.9,0.0,91,19.122,1.305630,8.422,1.195354,3.22,0.327109
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74487,2023-11-18,Y10ST0112,5.13,15.17,4.9,0.0,63,15.096,0.911773,4.950,0.159531,4.94,0.260768
74488,2023-11-19,Y10ST0112,4.68,14.52,4.9,0.0,64,14.964,0.943785,4.900,0.201122,5.00,0.187083
74489,2023-11-20,Y10ST0112,5.03,13.47,4.6,0.0,65,14.942,0.984515,4.900,0.201122,4.88,0.216795
74490,2023-11-21,Y10ST0112,4.71,12.67,4.0,0.0,66,14.278,1.197527,4.850,0.213190,4.72,0.454973


In [44]:
# Group milking_df by CowID and LactationNo
grouped = new_milking_df_pm.groupby(['CowID', 'LactationNo'])

# Initialize a list to store modified groups
final_groups = []
window_size = 5

# Iterate through each group
for (cowid, lactationno), group in grouped:

    group['Mean_Milk'] = group['Milk'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Milk'] = group['Milk'].rolling(window=window_size, min_periods=1).std()

    group['Mean_Duration'] = group['Duration'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Duration'] = group['Duration'].rolling(window=window_size, min_periods=1).std()

    group['Mean_Flow'] = group['PFlow'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Flow'] = group['PFlow'].rolling(window=window_size, min_periods=1).std()

    # delete the first five rows
    group = group.iloc[window_size-1:]
    final_groups.append(group)

# Combine all modified groups into a single DataFrame
final_milking_df_pm = pd.concat(final_groups)

final_milking_df_pm = final_milking_df_pm.drop('LacDay', axis=1)
final_milking_df_pm = final_milking_df_pm.drop('LactationNo', axis=1)
final_milking_df_pm = final_milking_df_pm.drop('Time', axis=1)
# final_milking_df_pm = final_milking_df_pm.drop('MilkDate', axis=1)
final_milking_df_pm = final_milking_df_pm.drop('index', axis=1)

final_milking_df_pm.reset_index(drop=True, inplace=True)

# Print the new DataFrame
final_milking_df_pm

,MilkDate,CowID,Duration,Milk,PFlow,Mastitis,DIM,Mean_Milk,Std_Milk,Mean_Duration,Std_Duration,Mean_Flow,Std_Flow
0,2022-01-05,208000988,5.93,11.57,3.4,0.0,87,10.550,1.472769,5.232,0.932829,3.38,0.148324
1,2022-01-06,208000988,5.53,10.85,3.4,0.0,88,10.400,1.373972,5.318,0.937427,3.34,0.089443
2,2022-01-07,208000988,4.90,10.67,3.6,0.0,89,10.312,1.330534,5.212,0.951457,3.40,0.141421
3,2022-01-08,208000988,4.08,9.90,3.4,0.0,90,10.680,0.612862,5.288,0.804282,3.40,0.141421
4,2022-01-09,208000988,4.93,11.28,3.5,0.0,91,10.854,0.639945,5.074,0.703513,3.46,0.089443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74629,2023-11-18,Y10ST0112,3.91,12.59,5.5,0.0,63,11.798,1.249068,4.138,0.553823,4.78,0.892749
74630,2023-11-19,Y10ST0112,4.13,9.62,4.1,0.0,64,11.186,1.443409,4.228,0.494136,4.50,0.827647
74631,2023-11-20,Y10ST0112,3.51,11.53,5.0,0.0,65,11.346,1.424440,4.098,0.592258,4.66,0.832466
74632,2023-11-21,Y10ST0112,4.08,10.60,4.3,0.0,66,11.432,1.346540,3.898,0.244479,4.82,0.597495


# Write to File

In [45]:
final_milking_df_pm['CowID'] = final_milking_df_pm['CowID'].astype(str)

if SAVE_DATA:
    final_milking_df_pm.to_csv("Sampling/binary/pm.csv")

final_milking_df_pm


,MilkDate,CowID,Duration,Milk,PFlow,Mastitis,DIM,Mean_Milk,Std_Milk,Mean_Duration,Std_Duration,Mean_Flow,Std_Flow
0,2022-01-05,208000988,5.93,11.57,3.4,0.0,87,10.550,1.472769,5.232,0.932829,3.38,0.148324
1,2022-01-06,208000988,5.53,10.85,3.4,0.0,88,10.400,1.373972,5.318,0.937427,3.34,0.089443
2,2022-01-07,208000988,4.90,10.67,3.6,0.0,89,10.312,1.330534,5.212,0.951457,3.40,0.141421
3,2022-01-08,208000988,4.08,9.90,3.4,0.0,90,10.680,0.612862,5.288,0.804282,3.40,0.141421
4,2022-01-09,208000988,4.93,11.28,3.5,0.0,91,10.854,0.639945,5.074,0.703513,3.46,0.089443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74629,2023-11-18,Y10ST0112,3.91,12.59,5.5,0.0,63,11.798,1.249068,4.138,0.553823,4.78,0.892749
74630,2023-11-19,Y10ST0112,4.13,9.62,4.1,0.0,64,11.186,1.443409,4.228,0.494136,4.50,0.827647
74631,2023-11-20,Y10ST0112,3.51,11.53,5.0,0.0,65,11.346,1.424440,4.098,0.592258,4.66,0.832466
74632,2023-11-21,Y10ST0112,4.08,10.60,4.3,0.0,66,11.432,1.346540,3.898,0.244479,4.82,0.597495


In [46]:
final_milking_df_am['CowID'] = final_milking_df_am['CowID'].astype(str)

if SAVE_DATA:
    final_milking_df_pm.to_csv("Sampling/binary/am.csv")

final_milking_df_am


,MilkDate,CowID,Duration,Milk,PFlow,Mastitis,DIM,Mean_Milk,Std_Milk,Mean_Duration,Std_Duration,Mean_Flow,Std_Flow
0,2022-01-05,208000988,7.28,16.94,3.0,0.0,87,18.458,2.052942,7.964,0.887542,3.10,0.070711
1,2022-01-06,208000988,8.41,20.27,3.7,0.0,88,18.386,1.962022,7.914,0.844529,3.20,0.282843
2,2022-01-07,208000988,8.08,19.93,3.4,0.0,89,18.428,1.999605,7.824,0.784302,3.26,0.288097
3,2022-01-08,208000988,7.91,19.06,3.1,0.0,90,18.382,1.977339,7.686,0.665304,3.26,0.288097
4,2022-01-09,208000988,10.43,19.41,2.9,0.0,91,19.122,1.305630,8.422,1.195354,3.22,0.327109
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74487,2023-11-18,Y10ST0112,5.13,15.17,4.9,0.0,63,15.096,0.911773,4.950,0.159531,4.94,0.260768
74488,2023-11-19,Y10ST0112,4.68,14.52,4.9,0.0,64,14.964,0.943785,4.900,0.201122,5.00,0.187083
74489,2023-11-20,Y10ST0112,5.03,13.47,4.6,0.0,65,14.942,0.984515,4.900,0.201122,4.88,0.216795
74490,2023-11-21,Y10ST0112,4.71,12.67,4.0,0.0,66,14.278,1.197527,4.850,0.213190,4.72,0.454973


# Alflex

### Combine all Alflex files

In [47]:
# List to store the combined dataframes
combined_df_list = []

# Iterate over each file in the directory
for filename in os.listdir(ALFLEX_PATH):
    if filename.endswith('.csv'):
        file_path = os.path.join(ALFLEX_PATH, filename)
        
        # Read the CSV file into a dataframe
        current_df = pd.read_csv(file_path)

        # Save only the selected columns
        current_df = current_df[['Cow Number', 'Raw Activity Data', 'Rumination Raw Data', 'Date', 'Time']]

        # Try parsing 'Date' column with different date formats
        try:
            current_df['Date'] = pd.to_datetime(current_df['Date'], format='%d/%m/%Y')
        except ValueError:
            # If the above format fails, try an alternative format
            current_df['Date'] = pd.to_datetime(current_df['Date'], format='%Y-%m-%d')
        
        current_df['Raw Activity Data'] = pd.to_numeric(current_df['Raw Activity Data'], errors='coerce')
        current_df['Rumination Raw Data'] = pd.to_numeric(current_df['Rumination Raw Data'], errors='coerce')
        current_df['Cow Number'] = current_df['Cow Number'].astype(int)

        # Combine the dataframe with the list
        combined_df_list.append(current_df)

# Combine all dataframes into a single dataframe
combined_df = pd.concat(combined_df_list, ignore_index=True)

# Sort by Cow Number
combined_df.sort_values(by='Cow Number', inplace=True)

# Sort within each group of Cow Number by Date and Time
combined_df.sort_values(by=['Cow Number', 'Date', 'Time'], inplace=True)

combined_df_cow_ids = combined_df["Cow Number"]
combined_df_cow_ids = combined_df_cow_ids.drop_duplicates()
# print("Number of Cows:", len(combined_df_cow_ids))

# Display the resulting combined and sorted dataframe
# combined_df


### Focus on Elinbank

In [48]:
# Get the list of CowNo from elinbank_stocklist_df
eligible_cow_numbers = elinbank_stocklist_df['CowNo'].tolist()

# Filter combined_df to keep only rows with Cow Number in eligible_cow_numbers
combined_df = combined_df[combined_df['Cow Number'].isin(eligible_cow_numbers)]

combined_df_cow_ids = combined_df["Cow Number"]
combined_df_cow_ids = combined_df_cow_ids.drop_duplicates()
print("Number of Cows:", len(combined_df_cow_ids))

# Display the filtered DataFrame
combined_df

Number of Cows: 729


,Cow Number,Raw Activity Data,Rumination Raw Data,Date,Time
2260548,441,NaN,NaN,2022-07-02,00:00
2260549,441,NaN,NaN,2022-07-02,02:00
2260550,441,NaN,NaN,2022-07-02,04:00
2260551,441,NaN,NaN,2022-07-02,06:00
2260552,441,NaN,NaN,2022-07-02,08:00
...,...,...,...,...,...
308395,9564,NaN,NaN,2022-03-31,14:00
308396,9564,NaN,NaN,2022-03-31,16:00
308397,9564,NaN,NaN,2022-03-31,18:00
308398,9564,NaN,NaN,2022-03-31,20:00


### Combine StockList to get CowID

In [49]:
# Merge dataframes on 'Animal Number' and 'CowNo'
combined_df = pd.merge(combined_df, elinbank_stocklist_df, left_on='Cow Number', right_on='CowNo', how='left')

# Drop unnecessary columns from the merged dataframe
combined_df.drop(['ShireFarm', 'CowNo', 'Dam', 'Shire', 'BirthDate'], axis=1, inplace=True)

# Display the updated dataframmerged_df
combined_df

,Cow Number,Raw Activity Data,Rumination Raw Data,Date,Time,CowID
0,441,NaN,NaN,2022-07-02,00:00,Y10ST0009
1,441,NaN,NaN,2022-07-02,02:00,Y10ST0009
2,441,NaN,NaN,2022-07-02,04:00,Y10ST0009
3,441,NaN,NaN,2022-07-02,06:00,Y10ST0009
4,441,NaN,NaN,2022-07-02,08:00,Y10ST0009
...,...,...,...,...,...,...
4493179,9564,NaN,NaN,2022-03-31,14:00,Y09ST0110
4493180,9564,NaN,NaN,2022-03-31,16:00,Y09ST0110
4493181,9564,NaN,NaN,2022-03-31,18:00,Y09ST0110
4493182,9564,NaN,NaN,2022-03-31,20:00,Y09ST0110


### Remove CowID with many nan

In [50]:
# Count the missing values for each Cow Number
missing_data_count = combined_df.groupby('Cow Number')['Raw Activity Data'].apply(lambda x: x.isnull().sum())

# Calculate the total number of entries for each Cow Number
total_entries = combined_df.groupby('Cow Number')['Raw Activity Data'].size()

# Calculate the missing data percentage
missing_data_percentage = (missing_data_count / total_entries) * 100

# Sort by missing data percentage in descending order
missing_data_percentage = missing_data_percentage.sort_values(ascending=False)

# Filter Cow Numbers with missing percentage higher than 50%
high_missing_percentage_cows = missing_data_percentage[missing_data_percentage > 30].index.tolist()

# Remove rows with Cow Numbers above the threshold
filtered_combined_df = combined_df[~combined_df['Cow Number'].isin(high_missing_percentage_cows)]

combined_df_cow_ids = filtered_combined_df["Cow Number"]
combined_df_cow_ids = combined_df_cow_ids.drop_duplicates()
print("Number of Cows:", len(combined_df_cow_ids))

filtered_combined_df

Number of Cows: 456


,Cow Number,Raw Activity Data,Rumination Raw Data,Date,Time,CowID
2244,552,26.0,86.0,2022-03-28,00:00,211012544
2245,552,27.0,87.0,2022-03-28,02:00,211012544
2246,552,26.0,77.0,2022-03-28,04:00,211012544
2247,552,48.0,59.0,2022-03-28,06:00,211012544
2248,552,65.0,11.0,2022-03-28,08:00,211012544
...,...,...,...,...,...,...
3055567,7634,78.0,0.0,2023-12-13,14:00,220030095
3055568,7634,52.0,0.0,2023-12-13,16:00,220030095
3055569,7634,57.0,1.0,2023-12-13,18:00,220030095
3055570,7634,48.0,31.0,2023-12-13,20:00,220030095


### Remove row with nan

In [51]:
# Remove rows with 0 or NaN values
filtered_combined_df = filtered_combined_df.replace(0, np.nan).dropna()

### Group by CowID and Date

In [52]:
# Group by CowID and Date and calculate the required statistics
alflex_df = filtered_combined_df.groupby(['CowID', 'Date']).agg(
    Alflex_Count=('Cow Number', 'count'),
    Activity=('Raw Activity Data', 'mean'),
    Rumination=('Rumination Raw Data', 'mean'),
    Min_Activity=('Raw Activity Data', 'min'),
    Min_Rumination=('Rumination Raw Data', 'min'),
    Max_Activity=('Raw Activity Data', 'max'),
    Max_Rumination=('Rumination Raw Data', 'max'),
).reset_index()

alflex_df

,CowID,Date,Alflex_Count,Activity,Rumination,Min_Activity,Min_Rumination,Max_Activity,Max_Rumination
0,208000988,2022-03-28,12,42.583333,42.750000,26.0,5.0,70.0,71.0
1,208000988,2022-03-29,12,41.333333,40.333333,25.0,5.0,60.0,67.0
2,208000988,2022-03-30,10,40.200000,44.300000,27.0,3.0,68.0,75.0
3,208000988,2022-03-31,11,41.272727,38.363636,27.0,5.0,61.0,67.0
4,208000988,2022-04-01,8,39.375000,45.250000,24.0,9.0,54.0,73.0
...,...,...,...,...,...,...,...,...,...
229848,Y10ST0117,2023-03-10,11,44.636364,40.363636,25.0,1.0,72.0,90.0
229849,Y10ST0117,2023-03-11,10,43.600000,40.800000,27.0,1.0,71.0,74.0
229850,Y10ST0117,2023-03-12,12,43.833333,35.500000,28.0,1.0,79.0,62.0
229851,Y10ST0117,2023-03-13,12,42.916667,46.333333,25.0,1.0,69.0,79.0


### Fill in missing date

In [53]:
# Group milking_df by CowID and LactationNo
grouped = alflex_df.groupby('CowID')

# Initialize a list to store modified groups
modified_groups = []

# Iterate through each group
for cowid, group in grouped:
    # Sort the group by MilkDate
    group = group.sort_values(by='Date')

    # Set MilkDate as the index
    group.set_index('Date', inplace=True)

    # Resample the group to fill in missing dates with a maximum gap of 5 days
    resampled_group = group.resample('D').ffill(limit=5)  # 會填5個

    # Reset the index to turn MilkDate back into a column
    resampled_group.reset_index(inplace=True)

    # Interpolate the new rows
    resampled_group['Alflex_Count'] = resampled_group['Alflex_Count'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Activity'] = resampled_group['Activity'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Rumination'] = resampled_group['Rumination'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Min_Activity'] = resampled_group['Min_Activity'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Min_Rumination'] = resampled_group['Min_Rumination'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Max_Activity'] = resampled_group['Max_Activity'].fillna(method='ffill').fillna(method='bfill')
    resampled_group['Max_Rumination'] = resampled_group['Max_Rumination'].fillna(method='ffill').fillna(method='bfill')

    # Workaround
    resampled_group.dropna(inplace=True)

    resampled_group.reset_index(inplace=True)

    # Remove rows where the difference between consecutive MilkDate values is greater than 5 days
    milkdate_diff = resampled_group['Date'].diff().dt.days
    milkdate_diff = milkdate_diff.fillna(1)
    rows_to_remove = milkdate_diff > 1
    if rows_to_remove.any():
        index_to_remove = rows_to_remove[rows_to_remove == True].index[0]  # Find the first index to remove
        resampled_group = resampled_group.iloc[:index_to_remove]  # Remove rows beyond the first index to remove

    # Append the modified group to the list
    modified_groups.append(resampled_group)

# Combine all modified groups into a single DataFrame
new_alflex_df = pd.concat(modified_groups)

# Print the new DataFrame
new_alflex_df


,index,Date,CowID,Alflex_Count,Activity,Rumination,Min_Activity,Min_Rumination,Max_Activity,Max_Rumination
0,0,2022-03-28,208000988,12.0,42.583333,42.750000,26.0,5.0,70.0,71.0
1,1,2022-03-29,208000988,12.0,41.333333,40.333333,25.0,5.0,60.0,67.0
2,2,2022-03-30,208000988,10.0,40.200000,44.300000,27.0,3.0,68.0,75.0
3,3,2022-03-31,208000988,11.0,41.272727,38.363636,27.0,5.0,61.0,67.0
4,4,2022-04-01,208000988,8.0,39.375000,45.250000,24.0,9.0,54.0,73.0
...,...,...,...,...,...,...,...,...,...,...
347,347,2023-03-10,Y10ST0117,11.0,44.636364,40.363636,25.0,1.0,72.0,90.0
348,348,2023-03-11,Y10ST0117,10.0,43.600000,40.800000,27.0,1.0,71.0,74.0
349,349,2023-03-12,Y10ST0117,12.0,43.833333,35.500000,28.0,1.0,79.0,62.0
350,350,2023-03-13,Y10ST0117,12.0,42.916667,46.333333,25.0,1.0,69.0,79.0


### Add five days mean, std

In [54]:
# Group milking_df by CowID and LactationNo
grouped = new_alflex_df.groupby('CowID')

# Initialize a list to store modified groups
final_groups = []
window_size = 5

# Iterate through each group
for cowid, group in grouped:

    group['Mean_Activity'] = group['Activity'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Activity'] = group['Activity'].rolling(window=window_size, min_periods=1).std()

    group['Mean_Rumination'] = group['Rumination'].rolling(window=window_size, min_periods=1).mean()
    group['Std_Rumination'] = group['Rumination'].rolling(window=window_size, min_periods=1).std()

    # delete the first five rows
    group = group.iloc[window_size-1:]
    final_groups.append(group)

# Combine all modified groups into a single DataFrame
final_alflex_df = pd.concat(final_groups)

final_alflex_df = final_alflex_df.drop('Alflex_Count', axis=1)
final_alflex_df = final_alflex_df.drop('index', axis=1)

final_alflex_df.reset_index(drop=True, inplace=True)

# Print the new DataFrame
final_alflex_df

,Date,CowID,Activity,Rumination,Min_Activity,Min_Rumination,Max_Activity,Max_Rumination,Mean_Activity,Std_Activity,Mean_Rumination,Std_Rumination
0,2022-04-01,208000988,39.375000,45.250000,24.0,9.0,54.0,73.0,40.952879,1.220856,42.199394,2.838007
1,2022-04-02,208000988,42.636364,36.454545,25.0,4.0,67.0,61.0,40.963485,1.238662,40.940303,3.774609
2,2022-04-03,208000988,41.375000,44.750000,26.0,31.0,64.0,58.0,40.971818,1.241908,41.823636,4.099838
3,2022-04-04,208000988,42.000000,38.500000,26.0,4.0,62.0,63.0,41.331818,1.222988,40.663636,4.044158
4,2022-04-05,208000988,41.142857,37.428571,25.0,20.0,57.0,56.0,41.305844,1.225932,40.476623,4.195880
...,...,...,...,...,...,...,...,...,...,...,...,...
191567,2023-03-10,Y10ST0117,44.636364,40.363636,25.0,1.0,72.0,90.0,41.074242,2.094107,41.559091,2.527786
191568,2023-03-11,Y10ST0117,43.600000,40.800000,27.0,1.0,71.0,74.0,41.660909,2.346997,41.152424,2.433334
191569,2023-03-12,Y10ST0117,43.833333,35.500000,28.0,1.0,79.0,62.0,42.263939,2.460924,39.216061,2.272158
191570,2023-03-13,Y10ST0117,42.916667,46.333333,25.0,1.0,69.0,79.0,42.830606,2.138343,40.766061,3.837106


# Combine Alflex and Milking

In [56]:
combined_milking_df_am = pd.merge(final_milking_df_am, final_alflex_df, left_on=['CowID', 'MilkDate'], right_on=['CowID', 'Date'], how='inner')

combined_milking_df_am = combined_milking_df_am.drop('Date', axis=1)
combined_milking_df_am = combined_milking_df_am.drop('CowID', axis=1)
combined_milking_df_am = combined_milking_df_am.drop('MilkDate', axis=1)
# combined_milking_df_am = combined_milking_df_am.drop('Alflex_Count', axis=1)

combined_milking_df_am

,Duration,Milk,PFlow,Mastitis,DIM,Mean_Milk,Std_Milk,Mean_Duration,Std_Duration,Mean_Flow,...,Activity,Rumination,Min_Activity,Min_Rumination,Max_Activity,Max_Rumination,Mean_Activity,Std_Activity,Mean_Rumination,Std_Rumination
0,6.88,14.15,3.1,0.0,173,13.498,0.690703,6.804,0.322614,2.88,...,39.375000,45.250000,24.0,9.0,54.0,73.0,40.952879,1.220856e+00,42.199394,2.838007
1,6.63,13.59,3.1,0.0,174,13.708,0.441158,6.864,0.225788,2.94,...,42.636364,36.454545,25.0,4.0,67.0,61.0,40.963485,1.238662e+00,40.940303,3.774609
2,6.70,13.14,2.8,0.0,175,13.518,0.440080,6.774,0.164712,2.90,...,41.375000,44.750000,26.0,31.0,64.0,58.0,40.971818,1.241908e+00,41.823636,4.099838
3,6.45,13.01,3.1,0.0,176,13.508,0.453453,6.662,0.154175,3.00,...,42.000000,38.500000,26.0,4.0,62.0,63.0,41.331818,1.222988e+00,40.663636,4.044158
4,6.81,11.97,2.8,0.0,177,13.172,0.806734,6.694,0.167123,2.98,...,41.142857,37.428571,25.0,20.0,57.0,56.0,41.305844,1.225932e+00,40.476623,4.195880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45402,5.03,13.58,5.2,0.0,60,14.386,0.658164,4.786,0.364870,5.04,...,43.583333,35.250000,25.0,1.0,71.0,72.0,42.990000,2.047783e+00,39.000000,4.057555
45403,4.96,15.99,4.8,0.0,61,14.734,0.959364,4.876,0.333961,4.90,...,43.583333,35.250000,25.0,1.0,71.0,72.0,43.790000,7.616320e-01,37.050000,2.495245
45404,4.70,15.56,5.2,0.0,62,15.046,0.913554,4.956,0.168315,4.94,...,43.583333,35.250000,25.0,1.0,71.0,72.0,43.886667,6.782740e-01,36.260000,2.258429
45405,5.13,15.17,4.9,0.0,63,15.096,0.911773,4.950,0.159531,4.94,...,43.583333,35.250000,25.0,1.0,71.0,72.0,43.583333,4.562530e-07,35.250000,0.000000


In [57]:
combined_milking_df_am.to_csv("Sampling/binary_alflex/am.csv", index=False)